<img src="cover.JPG">

In [ ]:
'''!pip install gensim'''

In [2]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
'''
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
'''

"\nnltk.download('punkt')\nnltk.download('wordnet')\nnltk.download('stopwords')"

In [4]:
#Import the data
csv=pd.read_csv("/SentimentAnalysis/Train.csv")

#Independent Variable
data=csv["text"]

In [5]:
data.head()

0    I grew up (b. 1965) watching and loving the Th...
1    When I put this movie in my DVD player, and sa...
2    Why do people who do not know what a particula...
3    Even though I have great interest in Biblical ...
4    Im a die hard Dads Army fan and nothing will e...
Name: text, dtype: object

In [6]:
#Stopwords except removing "not"
stopWord= stopwords.words('english')
stopWord.remove('not')

In [7]:
corpus=[]

#Removing Unnecessary Words
import re

for i in range(len(data)):
  review=re.sub('[^a-zA-Z]',' ',data[i])
  review= review.lower()
  review= review.split()
  review=[WordNetLemmatizer().lemmatize(word) for word in review if not word in set(stopWord)]
  review=' '.join(review)
  corpus.append(review)

In [32]:
corpus[0]

'grew b watching loving thunderbird mate school watched played thunderbird school lunch school wanted virgil scott one wanted alan counting became art form took child see movie hoping would get glimpse loved child bitterly disappointing high point snappy theme tune not could compare original score thunderbird thankfully early saturday morning one television channel still play rerun series gerry anderson wife created jonatha frakes hand director chair version completely hopeless waste film utter rubbish cgi remake may acceptable replacing marionette homo sapiens subsp sapiens huge error judgment'

In [9]:
#Create word tokens from Dictionary->x1,x2,..,xn
words=[]

import gensim
for sent in corpus:
  sentences=nltk.sent_tokenize(sent)
  for i in sentences:
    words.append(gensim.utils.simple_preprocess(i))

In [10]:
words

[['grew',
  'watching',
  'loving',
  'thunderbird',
  'mate',
  'school',
  'watched',
  'played',
  'thunderbird',
  'school',
  'lunch',
  'school',
  'wanted',
  'virgil',
  'scott',
  'one',
  'wanted',
  'alan',
  'counting',
  'became',
  'art',
  'form',
  'took',
  'child',
  'see',
  'movie',
  'hoping',
  'would',
  'get',
  'glimpse',
  'loved',
  'child',
  'bitterly',
  'disappointing',
  'high',
  'point',
  'snappy',
  'theme',
  'tune',
  'not',
  'could',
  'compare',
  'original',
  'score',
  'thunderbird',
  'thankfully',
  'early',
  'saturday',
  'morning',
  'one',
  'television',
  'channel',
  'still',
  'play',
  'rerun',
  'series',
  'gerry',
  'anderson',
  'wife',
  'created',
  'jonatha',
  'frakes',
  'hand',
  'director',
  'chair',
  'version',
  'completely',
  'hopeless',
  'waste',
  'film',
  'utter',
  'rubbish',
  'cgi',
  'remake',
  'may',
  'acceptable',
  'replacing',
  'marionette',
  'homo',
  'sapiens',
  'subsp',
  'sapiens',
  'huge',
 

## Word2Vec

In [11]:
from gensim.models import Word2Vec

model= Word2Vec(words,vector_size=100,window=5)

In [12]:
model.wv.index_to_key

['br',
 'movie',
 'film',
 'not',
 'one',
 'like',
 'time',
 'good',
 'character',
 'story',
 'even',
 'get',
 'would',
 'make',
 'see',
 'really',
 'well',
 'scene',
 'much',
 'bad',
 'people',
 'great',
 'also',
 'first',
 'show',
 'way',
 'thing',
 'made',
 'life',
 'could',
 'think',
 'go',
 'know',
 'watch',
 'love',
 'plot',
 'actor',
 'two',
 'seen',
 'many',
 'year',
 'end',
 'never',
 'say',
 'look',
 'acting',
 'best',
 'little',
 'ever',
 'man',
 'better',
 'take',
 'come',
 'work',
 'still',
 'part',
 'something',
 'director',
 'find',
 'back',
 'want',
 'give',
 'lot',
 'real',
 'guy',
 'watching',
 'performance',
 'woman',
 'play',
 'old',
 'though',
 'funny',
 'actually',
 'another',
 'role',
 'nothing',
 'going',
 'new',
 'every',
 'girl',
 'day',
 'world',
 'point',
 'horror',
 'cast',
 'feel',
 'minute',
 'fact',
 'thought',
 'comedy',
 'quite',
 'pretty',
 'around',
 'star',
 'seems',
 'action',
 'young',
 'big',
 'enough',
 'got',
 'however',
 'right',
 'long',
 'li

In [13]:
#Check for words similar to the given word
model.wv.similar_by_word('bad')

[('horrible', 0.7500149011611938),
 ('terrible', 0.7386503219604492),
 ('awful', 0.7139959931373596),
 ('suck', 0.7067276239395142),
 ('good', 0.695675253868103),
 ('lousy', 0.6624541282653809),
 ('crappy', 0.6422902345657349),
 ('atrocious', 0.6369722485542297),
 ('lame', 0.6268033981323242),
 ('laughable', 0.6068301200866699)]

In [14]:
#Average Word2Vec Approach
import numpy as np

def avgWord2Vec(input):
    return np.mean([model.wv[word] for word in input if word in model.wv.index_to_key],axis=0)


In [15]:
#Array of Independent Variable
x=[]
for i in range(len(words)):
    x.append(avgWord2Vec(words[i]))

x=np.array(x)

In [16]:
# Array of Dependent Variable
y=csv['label']

# Create a classifier

In [17]:
#Splitting
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [18]:
len(x_train)

28000

In [19]:
#Create a model using Support Vector Machine algorithm
from sklearn.svm import SVC

svm=SVC()

In [20]:
#Training the model
svm.fit(x_train,y_train)

SVC()

In [21]:
#Test the model
y_pred=svm.predict(x_test)

In [22]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [23]:
classification_report(y_test,y_pred)

'              precision    recall  f1-score   support\n\n           0       0.86      0.86      0.86      5926\n           1       0.87      0.87      0.87      6074\n\n    accuracy                           0.87     12000\n   macro avg       0.87      0.87      0.87     12000\nweighted avg       0.87      0.87      0.87     12000\n'

# CUSTOM REVIEW

In [39]:
#Input a review
new_review=input("Enter Review: ")

Enter Review: Formula movie about the illegitimate son of a rich Chilenian who stands to inherit a fortune and gets mixed up in the affairs of bad guys and falls in love with a beautiful female lawyer (Vargas). It looks very much like a TV movie, not really exciting. The only reason I bothered to see it was because Valentina Vargas was in it. No real surprises here, though it is nice to see Vargas. Great looking Chilenian landscapes on display but Malcolm McDowell's part is very small and doesn't add much to the movie. Michael Ironside plays as usual a bad guy but this is not one of his most memorable parts. The chase scenes are standard fare.


In [40]:
# Tokenzine then Remove the Unnecessary words
senten=nltk.sent_tokenize(new_review)
new_input=[]

for i in range(len(senten)):
    review=re.sub('[^a-zA-Z]',' ',senten[i])
    review=review.lower()
    review=review.split()
    review=[WordNetLemmatizer().lemmatize(word) for word in review if word not in set(stopWord)]
    review=" ".join(review)
    new_input.append(review)


In [41]:
#Create Word tokens from the custom review
new_words = []
for sent in new_input:
    sen = nltk.sent_tokenize(sent)
    for i in sen:
        new_words.append(gensim.utils.simple_preprocess(i))

new_words

[['formula',
  'movie',
  'illegitimate',
  'son',
  'rich',
  'chilenian',
  'stand',
  'inherit',
  'fortune',
  'get',
  'mixed',
  'affair',
  'bad',
  'guy',
  'fall',
  'love',
  'beautiful',
  'female',
  'lawyer',
  'vargas'],
 ['look', 'much', 'like', 'tv', 'movie', 'not', 'really', 'exciting'],
 ['reason', 'bothered', 'see', 'valentina', 'vargas'],
 ['real', 'surprise', 'though', 'nice', 'see', 'vargas'],
 ['great',
  'looking',
  'chilenian',
  'landscape',
  'display',
  'malcolm',
  'mcdowell',
  'part',
  'small',
  'add',
  'much',
  'movie'],
 ['michael',
  'ironside',
  'play',
  'usual',
  'bad',
  'guy',
  'not',
  'one',
  'memorable',
  'part'],
 ['chase', 'scene', 'standard', 'fare']]

In [42]:
# Words to Array with Average Word2Vec
new_x = []
for i in range(len(new_words)):
    new_x.append(avgWord2Vec(new_words[i]))

new_x = np.array(new_x)
new_x

array([[ 6.53376701e-05, -1.43927142e-01, -4.09890324e-01,
         3.85452360e-01, -2.53859550e-01, -2.23873943e-01,
         8.87692049e-02,  1.59742236e-01, -4.71020043e-02,
         2.42892385e-01,  3.83306593e-01, -8.63370374e-02,
         7.56570250e-02, -6.82897925e-01, -8.74410663e-03,
         1.90837055e-01, -3.90391231e-01, -8.10030758e-01,
        -2.79359192e-01, -4.16151166e-01,  2.75977291e-02,
         3.75354648e-01, -2.56317258e-01, -2.73255527e-01,
        -1.53720424e-01, -4.17574376e-01, -1.05266094e-01,
        -3.18204761e-01, -1.06167026e-01,  4.61964905e-01,
        -1.62315965e-01, -6.21272206e-01,  1.38448417e-01,
         8.59478340e-02, -7.90310979e-01,  3.77410829e-01,
        -2.49921069e-01, -8.33857238e-01, -2.48427138e-01,
        -1.32314622e-01,  1.25600696e-01, -2.57181019e-01,
         8.17945957e-01,  3.24167937e-01,  6.01893604e-01,
         4.17665184e-01, -2.19766334e-01, -1.71824291e-01,
         2.11735770e-01,  4.10793990e-01,  1.88501835e-0

In [43]:
# Predict using the SVM model
y_pred_new = svm.predict(new_x)
print(y_pred_new)

[0 0 0 1 1 0 0]


In [44]:
customreview=np.mean(y_pred_new)
if customreview < 0.6:print("Negative Review")
else:print("Positive Review")

Negative Review
